# Primera Parte
## Limpieza y procesamiento de datos

<img src="./logo_banguat_final.png">

Guatemala recibió en febrero de 2023 US$ 1,381.1 millones por remesasfamiliares, que equivale
a unos Q 10mil 910 millones. Las remesas son una fuente importante en la economía
guatemalteca, en el enlace: http://www.banguat.gob.gt/es/page/anos-2002-2023 podrá
encontrar un cuadro con el histórico de remesas familiares en Guatemala, cuadro que deberá
de normalizar respecto a la fecha para luego realizar las siguientes operaciones en columnas
aparte, tomar en cuenta que esta base de datos será actualizada recurrentemente.
- Crecimiento trimestral
- Crecimiento interanual
- Media móvil de 6 meses
Además de realizar una gráfica por cada columna (Valor, crecimientos, media móvil)
Detallar cada paso realizado en la operación, queries y formulas utilizadas.

### Librerias utilizadas para la limpieza de datos y obtencion de tabla principal

In [113]:
import openpyxl
import requests
import pandas as pd

### Consideracion de actualización de base de datos recurrentemente

Cómo la base de datos se va actualizando, no es viable la descarga directa y trabajar sobre el .xlsx sin actualizaciones. Necesitamos una forma de obtener siempre la última versión de este archivo y esto se logra solicitando el HTTP request directamente a la pagina y almacenando el archivo .xslx que nos envien localmente. Este proceso se realizará cada vez que se ejecute este Jupyter Notebook

In [114]:
# URL con el cual realizamos el HTTP request
url = "http://www.banguat.gob.gt/sites/default/files/banguat/estaeco/remesas/remfam2010_2023.xlsx"

# Obtencion del nombre
filename = url.split('/')[-1]
response = requests.get(url)

# Escritura del archivo localmente
with open(filename, 'wb') as f:
    f.write(response.content)

# Obtencion de df especifica del excel y de la worksheet
wb = openpyxl.load_workbook("remfam2010_2023.xlsx")
ws = wb['2002-2023']

# Valores en los que se encuentra la tabla que nos interesa
df = ws['B10':'X22']
rows = []
for row in df:
    rows.append([cell.value for cell in row])
df = pd.DataFrame(rows)

# Escribir el dfFrame en un archivo CSV
df.to_csv('remfam2010_2023_clean.csv', index=False, header=False)

# # Cargar el archivo CSV
df = pd.read_csv('remfam2010_2023_clean.csv', index_col=0)

# Realizar la suma en pandas ya que el valor devuelto del .xslx no es el valor sino la formula
total_row = df.sum(axis=0)
total_row.name = "Total"
df = pd.concat([df, total_row.to_frame().T])

# Guardar el archivo actualizado
df.to_csv('remfam2010_2023_clean.csv')
print("Archivo actualizado exitosamente.")

Archivo actualizado exitosamente.


### Leemos el archivo csv limpio

En este se encuentra nuestra tabla "Cuadro 1 Guatemala: Ingreso de Divisas por Remesas Familiares En millones de US dólares	Años: 2002 - 2023" que obtuvimos del .xlsx actualizable. Con esta información ya podemos empezar a realizar las operaciones que nos solicitan

In [115]:
df = pd.read_csv('remfam2010_2023_clean.csv', index_col=0)
print(df[['2002.0','2003.0','2004.0']])

                2002.0      2003.0      2004.0
Enero         83.15574   164.75655   194.74402
Febrero       96.65939   144.74341   183.73886
Marzo        109.03374   160.77811   227.89545
Abril        139.21199   181.08733   204.01319
Mayo          89.75174   187.37634   210.78039
Junio        127.97632   161.49457   212.12977
Julio        136.52537   191.95433   206.61293
Agosto       162.30790   178.91320   212.78174
Septiembre   157.33877   176.55097   211.77725
Octubre      176.26081   189.88092   218.02659
Noviembre    142.97583   173.37563   236.70772
Diciembre    158.19428   195.59340   231.41521
Total       1579.39188  2106.50476  2550.62312


### Normalización


In [116]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Cargar los datos del archivo CSV en un dfFrame de Pandas
df = pd.read_csv('remfam2010_2023_clean.csv', index_col=0)


"""Normalizacion respecto al valor final del año"""
df = df.div(df.iloc[-1], axis=1)
print(df[['2002.0','2003.0','2004.0']])

df.to_csv('remfam2010_2023_normalizado.csv')


"""Normalizacion con sklearn"""
# scaler = MinMaxScaler()
# df_norm = pd.DataFrame(scaler.fit_transform(df), columns=df.columns, index=df.index)

# df_norm.to_csv('remfam2010_2023_normalizado.csv')

# print(df_norm[['2002.0','2003.0','2004.0']])

              2002.0    2003.0    2004.0
Enero       0.052650  0.078213  0.076352
Febrero     0.061200  0.068713  0.072037
Marzo       0.069035  0.076325  0.089349
Abril       0.088143  0.085966  0.079986
Mayo        0.056827  0.088951  0.082639
Junio       0.081029  0.076665  0.083168
Julio       0.086442  0.091125  0.081005
Agosto      0.102766  0.084934  0.083423
Septiembre  0.099620  0.083812  0.083030
Octubre     0.111600  0.090140  0.085480
Noviembre   0.090526  0.082305  0.092804
Diciembre   0.100162  0.092852  0.090729
Total       1.000000  1.000000  1.000000


'Normalizacion con sklearn'

### Datos solicitados
Estos se encuentran dentro del archivo 'remfam2010_2023_datos2.csv' el cual luego de realizar las tazas solicitadas y agregarlas dentro del csv permite realizar las graficas. Para que esta implementación tome en cuenta datos actuales recopilados del .xslx mas reciente es necesario exportar los datos del Excel 'remfam2010_2023_datos.xlsx' a 'remfam2010_2023_datos2.xlsx' que es en donde se realizan las operaciones

In [117]:
# Convertir a archivo excel que nosotros modificaremos
df = pd.read_csv('remfam2010_2023_normalizado.csv')
df.to_excel('remfam2010_2023_datos.xlsx', index=False)
